# BEE 4750 Lab 3: Linear Programming with JuMP

**Name**: Priya Shah

**ID**: 5244508

> **Due Date**
>
> Wednesday, 10/16/24, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [31]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Downloads/BEE4750/labs/lab-3-ps742`


In [32]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver

## Overview

In this lab, you will write and solve a resource allocation example
using `JuMP.jl`. `JuMP.jl` provides an intuitive syntax for writing,
solving, and querying optimization problems.

`JuMP` requires the loading of a solver. \[Each supported solver works
for certain classes of problems, and some are open source while others
require a commercial license\]. We will use the `HiGHS` solver, which is
open source and works for linear, mixed integer linear, and quadratic
programs.

In this lab we will walk through the steps involved in coding a linear
program in HiGHS, solving it, and querying the solution.

## Exercise (3 points)

Your task is to decide how much lumber to produce to maximize profit
from wood sales. You can purchase wood from a managed forest, which
consists of spruce (320,000 bf) and fir (720,000 bf). Spruce costs
$\$0.12$ per bf to purchase and fir costs $\$0.08$ per bf.

At the lumber mill, wood can be turned into plywood of various grades
(see <a href="#tbl-inputs" class="quarto-xref">Table 1</a> for how much
wood of each type is required for and the revenue from each grade). Any
excess wood is sent to be recycled into particle board, which yields no
revenue for the mill.

| Plywood Grade | Inputs (bf/bf plywood) | Revenue (\$/1000 bf) |
|:-------------:|:----------------------:|:--------------------:|
|       1       |   0.5 (S) + 1.5 (F)    |         400          |
|       2       |   1.0 (S) + 2.0 (F)    |         520          |
|       3       |   1.5 (S) + 2.0 (F)    |         700          |

Table 1: Wood inputs and revenue by plywood grade. S refers to spruce
inputs, F fir inputs.

First, we need to identify our decision variables. While there are
several options, we will use $G_i$, the amount of each grade the mill
produces (in \$/1000 bf).

Using these decision variables, formulate a linear program to maximize
the profit of the mill subject to the supply constraints on spruce and
fir.

> **JuMP Syntax**
>
> The core pieces of setting up a `JuMP` model involve specifying the
> model and adding variables, the objective, and constraints. At the
> most simple level, this syntax looks like this:
>
> ``` julia
> m = Model(HiGHS.Optimizer)
> @variable(m, lb <= x <= ub) # if you do not have upper or lower bounds, you can drop those accordingly
> @variable(m, lb <= y <= ub)
> @objective(m, Max, 100x + 250y) # replace Max with Min depending on the problem
> @constraint(m, label, 6x + 12y <= 80) # replace "label" with some meaningful string you would like to use later to query shadow prices, or drop it
> ```
>
> You can add more constraints or more variables as needed.

> **Using Array Syntax**
>
> You can set up multiple variables or constraints at once using array
> syntax. For example, the following are equivalent:
>
> ``` julia
> @variable(m, G1 >= 0)
> @variable(m, G2 >= 0)
> @variable(m, G3 >= 0)
> ```
>
> and
>
> ``` julia
> @variable(m, G[1:3] >= 0)
> ```
>
> You can also set up multiple constraints using arrays of coefficients
> and/or bounds. For example:
>
> ``` julia
> I = 1:3
> d = [0; 3; 5]
> @constraint(m, demand[i in I], G[i] >= d[i])
> ```

`JuMP` is finicky about changing objects and constraints, so I recommend
setting up all of the model syntax in one notebook cell, which is what
we will do here.

**Derviations**

***Objective Function:***
Maximize the profit, which is the total revenue minus the cost of the wood. 

The revenues are given as:
- G1: $400 per 1000 bf
- G2: $520 per 1000 bf
- G3: $700 per 1000 bf

Therefore the equation for revenue can be written as: $Revenue=400G[1]+520G[2]+700G[3]$

The costs are associated with the type of wood used:
- Spruce (S): $0.12 per bf, which translates to $120 per 1000 bf
- Fir (F): $0.08 per bf, which translates to $80 per 1000 bf

Therefore the equation for cost can be written as: $Cost=120(0.5G[1]+1.0G[2]+1.5G[3])+80(1.5G[1]+2.0G[2]+2.0G[3])$

The total profit function is then:

$Profit=Revenue from plywood−Cost of wood$

The objective function is to maximize this profit, which is in units of $/1000 bf:

$ Maximize (400G[1]+520G[2]+700G[3]-(120(0.5G[1]+1.0G[2]+1.5G[3])+80(1.5G[1]+2.0G[2]+2.0G[3])))$

***Constraints:***

The mill has limitations on the available spruce and fir:
- Available spruce: 320,000 bf, which translates to 320 (1000 bf)
- Available fir: 720,000 bf, which translates to 720 (1000 bf)

These translate into constraints on the usage of spruce and fir: 

Spruce constraint: $0≤0.5G[1]+1.0G[2]+1.5G[3]≤320$

Fir constraint: $0≤1.5G[1]+2.0G[2]+2.0G[3]≤720$

The non-negativity constraints are implemented through the definition of the variable where G[1:3] >= 0

In [33]:
forest_model = Model(HiGHS.Optimizer) # initialize model object
@variable(forest_model, G[1:3] >= 0) # non-negativity constraints
# uncomment the following lines and add the objective and constraints as needed for the model
@objective(forest_model, Max, (400*G[1]) + (520*G[2]) + (700*G[3]) - (120 * (0.5G[1] + 1.0G[2] + 1.5G[3]) + 80 * (1.5G[1] + 2.0G[2] + 2.0G[3])))
@constraint(forest_model, spruce_constraint, 0.5G[1] + 1.0G[2] + 1.5G[3] <= 320) 
@constraint(forest_model, fir_constraint, 1.5G[1] + 2.0G[2] + 2.0G[3] <= 720)
print(forest_model) # this outputs a nicely formatted summary of the model so you can check your specification

Max 220 G[1] + 240 G[2] + 360 G[3]
Subject to
 spruce_constraint : 0.5 G[1] + G[2] + 1.5 G[3] ≤ 320
 fir_constraint : 1.5 G[1] + 2 G[2] + 2 G[3] ≤ 720
 G[1] ≥ 0
 G[2] ≥ 0
 G[3] ≥ 0


Next, to optimize, use the `optimize!()` function:

In [34]:
optimize!(forest_model)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-01, 2e+00]
  Cost   [2e+02, 4e+02]
  Bound  [0e+00, 0e+00]
  RHS    [3e+02, 7e+02]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 3(-0); elements 6(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -8.1999929744e+02 Ph1: 2(8.5); Du: 3(819.999) 0s
          2     1.1200000000e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.1200000000e+05
HiGHS run time      :          0.00


You should get confirmation that a solution was found; if one was not,
there’s a chance something was wrong with your model formulation.

To find the values of the decision variables, use `value()` (which can
be broadcasted over variable arrays):

In [35]:
@show value.(G);

value.(G) = [352.0, 0.0, 96.0]


Similarly, `objective_value()` finds the optimal value of the objective:

In [36]:
@show objective_value(forest_model);

objective_value(forest_model) = 112000.0


Finally, we can find the dual values of the constraints with
`shadow_price()`. Do this for the constraints in your model using the
block below.

In [37]:
#@show shadow_price(name_of_constraint);

@show shadow_price(spruce_constraint)
@show shadow_price(fir_constraint)

shadow_price(spruce_constraint) = 80.0
shadow_price(fir_constraint) = 120.0


120.0

`JuMP` also lets you evaluate other expressions that you might be
interested in based on the solutions. For example, you can use the
following block to calculate the total amount of plywood the mill would
produce under the optimal solution:

In [38]:
@expression(forest_model, total_plywood, sum(G))
@show value.(total_plywood);

value.(total_plywood) = 448.0


**Solution:**

Based on this optimization, I found the following:

- The optimal amount of each wood grade should be: **352** thousand bf of grade 1, **0** bf of grade 2, and **96** thousand bf of grade 3
    - The means the total plywood is **448** thousand bf
- This generates a profit of **$112000**
- The shadow price of the constraints show how much profit would increase for each additional bf of wood and whether or not the contraint is binding. 
    - Shadow price of spruce: **$80**. This constraint is binding. 
    - Shadow price of fir: **$120**. This constraint is binding. 

## References

Classmates referenced:
- Bailey Belinger
- Camila Monter